Model page: https://api-inference.huggingface.co/models/LazarusNLP/all-indo-e5-small-v4



In [ ]:
# ================== INSTALL (hanya sekali) ==================
!pip install flask flask-cors pyngrok requests numpy scikit-learn --quiet

# ================== CONFIGURATION ==================
HF_TOKEN = "hf_token here"  # Ganti dengan HuggingFace token kamu
NGROK_AUTH_TOKEN = "token here"  # Ganti dengan Ngrok token kamu

# ================== IMPORTS ==================
import requests
from flask import Flask, request, jsonify
from flask_cors import CORS
import numpy as np
import json
from pyngrok import ngrok
from threading import Thread

# ================== HuggingFace API Config ==================
API_URL = "https://api-inference.huggingface.co/models/LazarusNLP/all-indo-e5-small-v4"
HEADERS = {"Authorization": f"Bearer {HF_TOKEN}"}

# ================== Sentence Similarity ==================
def get_similarity_scores_from_api(source_sentence, candidate_sentences):
    payload = {
        "inputs": {
            "source_sentence": source_sentence,
            "sentences": candidate_sentences
        }
    }
    try:
        response = requests.post(API_URL, headers=HEADERS, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ Error HuggingFace: {e}")
        return None

# ================== Product Recommender ==================
def recommend_products(user_query, product_database, top_n=3):
    candidate_sentences = []
    index_map = []

    for idx, p in enumerate(product_database):
        if p.get("search_vector") and p["search_vector"] != "null":
            candidate_sentences.append(p["search_vector"])
            index_map.append(idx)
        elif p.get("description"):
            candidate_sentences.append(p["description"])
            index_map.append(idx)
        else:
            candidate_sentences.append(p["name"])
            index_map.append(idx)

    print(f"\n[DEBUG] Total kandidat: {len(candidate_sentences)}")
    print(f"[DEBUG] Kalimat kandidat: {candidate_sentences}")

    similarity_scores = get_similarity_scores_from_api(user_query, candidate_sentences)
    if similarity_scores is None:
        return []

    print(f"[DEBUG] Similarity scores: {similarity_scores}")

    similarity_scores = np.array(similarity_scores)
    ranked_indices = np.argsort(similarity_scores)[::-1]

    recommendations = []
    for i in range(min(top_n, len(ranked_indices))):
        idx = ranked_indices[i]
        original_idx = index_map[idx]
        product = product_database[original_idx].copy()
        product["similarity_score"] = float(similarity_scores[idx])
        recommendations.append(product)

    print(f"[DEBUG] Total rekomendasi: {len(recommendations)}\n")
    return recommendations

# ================== Flask API ==================
app = Flask(__name__)
CORS(app)

@app.route('/')
def home():
    return "✅ AI Product Recommender API is running!"

@app.route('/recommend', methods=['POST'])
def get_recommendations():
    data = request.get_json()
    user_query = data.get("userQuery")
    products = data.get("products")

    if not user_query or not products:
        return jsonify({"error": "❌ Missing userQuery or products"}), 400

    recommended = recommend_products(user_query, products, top_n=3)
    return jsonify({
        "data": recommended,
        "message": "✅ Rekomendasi produk berhasil diambil"
    }), 200

# ================== Run Flask in Background ==================
def run_flask():
    app.run(host="0.0.0.0", port=5000, debug=False)

flask_thread = Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# ================== Ngrok Tunnel ==================
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(5000).public_url
print(f"\n🚀 Ngrok URL: {public_url}/recommend")
print("✅ API sudah siap! Coba POST pakai Postman/curl.")


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit



🚀 Ngrok URL: https://f82a-34-142-237-143.ngrok-free.app/recommend
✅ API sudah siap! Coba POST pakai Postman/curl.
